<a href="https://colab.research.google.com/github/WVU-AIClub/Custom-TTS/blob/Sean-Branch/Piper/Piper_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0) Colab runtime + GPU check

In [11]:
%cd ~/Custom-TTS/   # This must be the root folder for the colab

# Hyperparams and Folder path setup
voice_folder = "Peter"
VOICE_NAME = "PETAH"

ESPEAK_VOICE    = "en-us"  # run `!espeak-ng --voices` to see options
SAMPLE_RATE_HZ  = 22050 # DEFAULT: 22050
BATCH_SIZE      = 4       # drop to 8 or 4 if you OOM

DATA_ROOT       = f"{voice_folder}"
AUDIO_DIR       = DATA_ROOT + "/outputs"
CSV_PATH        = DATA_ROOT + "/metadata.csv"

CACHE_DIR       = f"{voice_folder}/piper_cache"
CONFIG_PATH     = DATA_ROOT + f"/{VOICE_NAME}.json"

# Optional: start from an existing checkpoint to speed up & stabilize training
# Get a .ckpt from https://huggingface.co/datasets/rhasspy/piper-checkpoints (medium quality recommended)
CKPT_PATH       = "medium.ckpt"

# Make sure dirs exist
import os
os.makedirs(CACHE_DIR, exist_ok=True)
print("CSV exists:", os.path.exists(CSV_PATH))
print("Audio dir exists:", os.path.exists(AUDIO_DIR))
print("Cache dir:", CACHE_DIR)
print("Config will be written to:", CONFIG_PATH)

[Errno 2] No such file or directory: '/home/aiwvu/Custom-TTS/ # This must be the root folder for the colab'
/home/aiwvu/Custom-TTS/piper1-gpl
CSV exists: False
Audio dir exists: False
Cache dir: Peter/piper_cache
Config will be written to: Peter/PETAH.json


/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [12]:
# Finds the most recent checkpoint
%cd ~/Custom-TTS/
import os, glob

version_dirs = glob.glob('lightning_logs/version_*')
if not version_dirs:
    raise FileNotFoundError("No version directories found in 'lightning_logs'")
latest_version_dir = max(version_dirs, key=lambda d: int(d.split("version_")[-1]))
print(f"Selected Version Directory: {latest_version_dir}")

ckpt_dir = os.path.join(latest_version_dir, "checkpoints")
CKPT_PATH = glob.glob(f"{ckpt_dir}/*.ckpt")[0]

# The should only ever be 1 checkpoint
if not CKPT_PATH:
  raise FileNotFoundError(f"No .ckpt files found in {ckpt_dir}")
print(CKPT_PATH)

/home/aiwvu/Custom-TTS
Selected Version Directory: lightning_logs/version_5
lightning_logs/version_5/checkpoints/epoch=5347-step=25464.ckpt


/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
# Check CSV file for missing transcripts
import pandas as pd
csv_path = str(CSV_PATH)
if os.path.exists(csv_path):
    # Read as pipe-delimited, two columns
    try:
        df = pd.read_csv(csv_path, sep="|", header=None, names=["audio","text"])
        print(df.head())
        # Check a few audio files exist
        missing = [a for a in df["audio"].head(5) if not (AUDIO_DIR/str(a)).exists()]
        print("Missing among first 5:", missing)
    except Exception as e:
        print("CSV read error:", e)
else:
    print("CSV not found at:", csv_path)

   audio                                               text
0  1.wav  Harris got the job on how I met your mother be...
1  2.wav  Well, I was doing the dishes for a while, but ...
2  3.wav  dog kiss? Oh sounds like, uh sounds like dog k...
3  4.wav  I just don't know why you said it then. Mmm, t...
4  5.wav  call my mother, I'm gonna have to pinch you fo...
CSV read error: unsupported operand type(s) for /: 'str' and 'str'


In [14]:
# Check GPU
import torch, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

Python: 3.10.12
PyTorch: 2.5.1+cu124
CUDA available: True
Sun Nov 23 17:49:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8              5W /   80W |      34MiB /   8188MiB |      0%      Default |
|                                         |                        |              

1) System packages (incl. eSpeak dev)

In [15]:
# If not running on Google Colab. Run this yourself
# !sudo apt-get update -y
# !sudo apt-get install -y build-essential cmake ninja-build espeak-ng espeak-ng-data libespeak-ng-dev pkg-config ffmpeg
# !pkg-config --modversion espeak-ng

2) Clone repo fresh

In [16]:
# Clone repo (Only run if voice solution is fixed. See more in inference.py)
# !rm -rf piper1-gpl
# !git clone https://github.com/OHF-voice/piper1-gpl.git
# %cd piper1-gpl
# !pwd

In [17]:
%%capture
%cd ~/Custom-TTS/piper1-gpl

# Python deps (editable install)
!python3 script/dev_build
!python3 -m pip install --upgrade pip setuptools wheel scikit-build cmake ninja
!python3 -m pip install -e ".[train]"

In [18]:
# Build the Cython extension used for alignment
!./build_monotonic_align.sh
!python3 setup.py build_ext --inplace -v

Compiling /home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx because it changed.
[1/1] Cythonizing /home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/monotonic_align/core.pyx
performance hint: core.pyx:5:0: Exception check on 'maximum_path_each' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_each' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_each' to allow an error code to be returned.
performance hint: core.pyx:36:0: Exception check on 'maximum_path_c' will always require the GIL to be acquired.
Possible solutions:
	1. Declare 'maximum_path_c' as 'noexcept' if you control the definition and you're sure you don't want the function to raise exceptions.
	2. Use an 'int' return type on 'maximum_path_c' to allow an error code to be returned.
performance hint: core.pyx:42:21: Exception check after calling 'm

Kick off training

In [19]:
%cd ~/Custom-TTS/
import subprocess
train_cmd = [
    "python3", "-m", 'piper.train', 'fit',
  "--data.voice_name", VOICE_NAME,
  "--data.csv_path", CSV_PATH,
  "--data.audio_dir", AUDIO_DIR,
  "--model.sample_rate", str(SAMPLE_RATE_HZ),
  "--data.espeak_voice", f"{ESPEAK_VOICE}",
  "--data.cache_dir", CACHE_DIR,
  "--data.config_path", CONFIG_PATH,
  "--data.batch_size", str(BATCH_SIZE),
  "--ckpt_path", CKPT_PATH
]

subprocess.run(train_cmd, check=True)

/home/aiwvu/Custom-TTS


/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/lightning/fabric/utilities/seed.py:44: No seed found, seed set to 0
Seed set to 0
/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
INFO:piper.train.vits.dataset:Processed 21 utterance(s)
Restoring states from the checkpoint path at lightning_logs/version_5/checkpoints/epoch=5347-step=25464.ckpt
/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:445: The dirpath has changed from '/home/aiwvu/Custom-TT

Epoch 5348:   0%|          | 0/4 [00:00<?, ?it/s]                          

/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=21` in the `DataLoader` to improve performance.
/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
/home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/dataset.py:477: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted

Epoch 5348: 100%|██████████| 4/4 [00:02<00:00,  1.67it/s, v_num=6]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/dataset.py:477: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  phoneme_ids=torch.load(utt.phoneme_ids_path),
/home/aiwv


Epoch 5349: 100%|██████████| 4/4 [00:02<00:00,  1.94it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5350: 100%|██████████| 4/4 [00:02<00:00,  1.91it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5351: 100%|██████████| 4/4 [00:02<00:00,  1.90it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5352: 100%|██████████| 4/4 [00:02<00:00,  1.93it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5353: 100%|██████████| 4/4 [00:02<00:00,  1.91it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5354: 100%|██████████| 4/4 [00:02<00:00,  1.91it/s, v_num=6]    
Validation: |          | 0/? [00:00<?, ?it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 5355: 100%|████


Detected KeyboardInterrupt, attempting graceful shutdown ...


KeyboardInterrupt: 

In [20]:
# Finds the most recent checkpoint
%cd ~/Custom-TTS/
import os, glob

version_dirs = glob.glob('lightning_logs/version_*')
if not version_dirs:
    raise FileNotFoundError("No version directories found in 'lightning_logs'")
latest_version_dir = max(version_dirs, key=lambda d: int(d.split("version_")[-1]))
print(f"Selected Version Directory: {latest_version_dir}")

ckpt_dir = os.path.join(latest_version_dir, "checkpoints")
CKPT_PATH = glob.glob(f"{ckpt_dir}/*.ckpt")[0]

# The should only ever be 1 checkpoint
if not CKPT_PATH:
  raise FileNotFoundError(f"No .ckpt files found in {ckpt_dir}")
print(CKPT_PATH)

/home/aiwvu/Custom-TTS
Selected Version Directory: lightning_logs/version_6
lightning_logs/version_6/checkpoints/epoch=6014-step=30800.ckpt


/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


10. Export to onnx

In [21]:
export_cmd = [
    sys.executable, "-m", "piper.train.export_onnx",
  "--checkpoint", CKPT_PATH,
  "--output-file", f"{voice_folder}/{VOICE_NAME}.onnx"
]
subprocess.run(export_cmd, check=True)

/home/aiwvu/Custom-TTS/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/attentions.py:235: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  t_s == t_t
/home/aiwvu/Custom-TTS/piper1-gpl/src/piper/train/vits/attentions.py:295: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  pad_length = max(length - (self.window_size + 1), 0)
/home/

CompletedProcess(args=['/home/aiwvu/Custom-TTS/.venv/bin/python', '-m', 'piper.train.export_onnx', '--checkpoint', 'lightning_logs/version_6/checkpoints/epoch=6014-step=30800.ckpt', '--output-file', 'Peter/PETAH.onnx'], returncode=0)